In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import InMemoryRunner
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext, FunctionTool
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.models.google_llm import Gemini

In [3]:
import APIKey    #Storing my API key in python class "APIKey"
import os

google_API = APIKey.GOOGLESTUDIO_API_KEY    #My google studio API

os.environ["GOOGLE_API_KEY"] = google_API

In [4]:
import yfinance as yf

In [5]:
google_news_agent = Agent(
    name="GoogleNewsAgent",
    model="gemini-2.5-flash",
    instruction="""You are a specialized news arrangement agent for a given company. 
    
    For a given company:
    1. Use 'google_search' to find the company's summary.
    2. Read the company's summary and then use 'google_search' to find current related news that would affect the company's stock price.
    3. Generate comment with important points and analysis about the company's stock future performance base on the news.
    """,
    tools = [google_search],
    output_key="google_news_arrangement",
)

In [6]:
def competitors_compare(target: str) -> dict:
    """Searching for the target company's main competitors and compare their main financial ratio.

    Args:
        taregt: The stock ticker of a company. It should be consisted of uppercase letters,
                e.g., "AAPL" or "NVDA".

    Returns:
        Dictionary with status and rate information.
        Success: {"status": "success", "comparison":
        | Ticker   |   Dividend Yield |   Trailing PE |   TTM PS |   Profit Margin |   PB Ratio |   
        Trailing EPS |   EV/EBITDA |   Current Ratio |   Debt-to-Equity |    ROA |    ROE |   PEG Ratio |\n|:---------|-----------------:|
        --------------:|---------:|----------------:|-----------:|---------------:|------------:|----------------:|-----------------:|-------:|
        -------:|------------:|\n| SHEL     |             3.8  |       15.3694 |   0.8023 |          0.0544 |     2.4892 |           4.9  |
        9.868 |           1.352 |           41.602 | 0.049  | 0.0815 |      1.634  |\n| XOM      |             3.5  |       17.3009 |   1.5554 |
        0.0918 |     1.9265 |           6.88 |       8.626 |           1.137 |           15.672 | 0.0511 | 0.1142 |      2.7756 |\n| CVX      |
        4.42 |       21.6062 |   1.6404 |          0.0677 |     1.6293 |           7.11 |       9.323 |           1.152 |           21.239 |
        0.0405 | 0.0732 |      2.9249 |\n| NFG      |             2.66 |       14.3187 |   3.2274 |          0.2277 |     2.3753 |           5.68 |
        7.132 |           0.444 |           91.548 | 0.0701 | 0.1745 |    N/A      |\n| UNTC     |            15.08 |        4.9303 |   1.3284 |
        0.2334 |     1.3559 |           6.74 |       2.907 |           3.545 |            1.009 | 0.15   | 0.2694 |    N/A      |}
        Error: {"status": "error", "error_message": "Stock ticker not found"}
    """

    metrics = ['dividendYield', 'trailingPE', 'priceToSalesTrailing12Months', 'profitMargins', 'priceToBook', 'trailingEps', 'enterpriseToEbitda', 
    'currentRatio', 'debtToEquity', 'returnOnAssets', 'returnOnEquity', 'trailingPegRatio']
    col_names = ['Ticker', 'Dividend Yield', 'Trailing PE', 'TTM PS', 'Profit Margin', 'PB Ratio', 'Trailing EPS', 'EV/EBITDA', 'Current Ratio',
               'Debt-to-Equity', 'ROA', 'ROE', 'PEG Ratio']
    
    ticker = yf.Ticker(target)
    info = ticker.info
    compare_df = pd.DataFrame(columns = col_names)
    
    industry = yf.Industry(info['industryKey'])
    competitors = list(industry.top_companies.index.values)
    competitors.insert(0, target)
    
    for comp in competitors:
        comp_info = yf.Ticker(comp).info
    
        row_data = [comp]
    
        for metric in metrics:
            value = comp_info.get(metric, 'NaN') 
            row_data.append(value)
    
        compare_df.loc[len(compare_df)] = row_data

    compare_df = compare_df.replace("NaN", np.nan)
    compare_df = compare_df.dropna(axis=1, how='all')
    compare_df = compare_df.fillna("N/A")

    compare_df = compare_df.round(4)
    compare_df_string = compare_df.to_markdown(index=False)

    if compare_df_string is not None:
        return {"status": "success", "comparison": compare_df_string}
    else:
        return {"status": "error", "error_message": f"Stock ticker '{target}' not found"}

In [7]:
competitors_agent = Agent(
    name="CompetitorsAgent",
    model="gemini-2.5-flash",
    instruction="""You are a specialized agent to compare the target company to its competitors.

    For a given company:
    1. Use 'competitors_compare' to get the target company and its main competitors' financial ratio data.
    2. Check the "status" field in it response for errors.
    3. Aanlyse the financial ratio data, summarize the most important points and provide some insights that might affect the target companies' 
    stock price.
    
    If any tool returns status "error", explain the issue to the user clearly.""",
    tools = [competitors_compare],
    output_key="comparing_competitors",
)

In [8]:
aggregator_agent = Agent(
    name="AggregatorAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Combine these two analyses into a single investment memo:

    **News:**
    {google_news_arrangement}
    
    **Comparison:**
    {comparing_competitors}
    
    Your memo should highlight influences to the target stock price, recommendations, and the risk from these two analyses. The final summary should be around 200 words.""",
    output_key="investment_memo", # This will be the final output of the entire system.
)

In [9]:
parallel_analysis_team = ParallelAgent(
    name="ParallelAnalysisTeam",
    sub_agents=[google_news_agent, competitors_agent],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="AnalysisSystem",
    sub_agents=[parallel_analysis_team, aggregator_agent],
)

In [10]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("SHEL", verbose = True)


 ### Created new session: debug_session_id

User > SHEL


CompetitorsAgent > [Calling tool: competitors_compare({'target': 'SHEL'})]


C:\Users\User\AppData\Local\Temp\ipykernel_27472\337580190.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  compare_df.loc[len(compare_df)] = row_data
C:\Users\User\AppData\Local\Temp\ipykernel_27472\337580190.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  compare_df.loc[len(compare_df)] = row_data
C:\Users\User\AppData\Local\Temp\ipykernel_27472\337580190.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this

CompetitorsAgent > [Tool result: {'status': 'success', 'comparison': '| Ticker   |   Dividend Yield | Trailing PE   |   TTM PS |   Pr...]
CompetitorsAgent > SHEL's financial ratios present a mixed but insightful picture compared to its competitors.

**Key Strengths:**
*   **Valuation:** SHEL appears to be relatively undervalued based on its **Trailing PE (15.09)** and **TTM PS (0.7856)**, which are lower than major competitors like ExxonMobil (XOM) and Chevron (CVX). Its **PEG Ratio (1.6122)** is also more favorable than XOM and CVX, suggesting it might be reasonably priced relative to its growth prospects.
*   **Liquidity:** SHEL boasts a healthy **Current Ratio (1.352)**, indicating good short-term financial health and ability to cover its immediate liabilities.

**Areas for Improvement:**
*   **Profitability and Efficiency:** SHEL's **Profit Margin (0.0544)**, **Trailing EPS (4.9)**, **ROA (0.049)**, and **ROE (0.0815)** are generally lower than several peers such as XOM, NFG, and U